In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
import pytorch_lightning as pl
from DEQLighting import DEQFixedPoint,LinearBlock,anderson
import torch.nn as nn

In [ ]:
class MyDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.data=pd.read_excel('Dry_Bean_Dataset.xlsx',index_col=None)
        self.data['Class']=self.data['Class'].astype('category')
        cat_columns = self.data.select_dtypes(['category']).columns
        self.data[cat_columns] = self.data[cat_columns].apply(lambda x: x.cat.codes)
    def __len__(self) -> int:
        return len(self.data)
    def __getitem__(self, index: int):
        return (torch.from_numpy(self.data.loc[index,self.data.columns!='Class'].to_numpy()).float(),torch.from_numpy(np.asarray([self.data.loc[index,'Class']])).squeeze().long())

In [ ]:
class MyDEQLayer(nn.Module):
    def __init__(self):
        super(MyDEQLayer,self).__init__()
        self.lb1=LinearBlock(256,512,True)
        self.lb2=LinearBlock(512,512,True)
        self.lb3=LinearBlock(512,512,True)
        self.lb4=LinearBlock(512,256,True)
        self.lb5=LinearBlock(256,256,True)
    def forward(self,z,x,skip=None):
        curr=self.lb1(z)
        curr=self.lb2(curr)
        curr=self.lb3(curr)
        curr=self.lb4(curr)
        curr=self.lb5(curr+x)

        return curr

class MyModel(pl.LightningModule):
    def __init__(self,batch_size,lr):
        super().__init__()
        self.encoder=LinearBlock(16,256,True)
        self.deq=DEQFixedPoint(MyDEQLayer(),anderson,tol=1e-4, max_iter=50, m=5)
        #self.lb1=LinearBlock(256,256,True)
        #self.lb2=LinearBlock(256,256,True)
        #self.lb3=LinearBlock(256,256,True)
        self.decoder=nn.Sequential(nn.Linear(256,7))
        self.entropy=nn.CrossEntropyLoss()
        self.batch_size=batch_size
        self.lr=lr
    def forward(self,x):
        curr=self.encoder(x)
        curr=self.deq(curr,None)
        return self.decoder(curr)
    def training_step(self,batch,batch_idx):
        X,y=batch
        pred=self(X)
        loss=self.entropy(pred,y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 2000*13612, eta_min=1e-6),
                    "monitor": "train_loss",
                    "frequency": 1,
                    "interval": "epoch"
                    }
                }
    def train_dataloader(self):
        trainLoader=DataLoader(MyDataset(),num_workers=8,pin_memory=True,batch_size=self.batch_size,shuffle=True)
        return trainLoader


In [ ]:
model=MyModel(10000,0.01)

In [ ]:
trainer=pl.Trainer(gpus=1,log_every_n_steps=1,max_epochs=2000)

In [ ]:
trainer.fit(model)